In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import cred_pg as c

In [2]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=usozmed,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [3]:
def exe(sql):
    with engine.connect() as con:
        return pd.read_sql_query(text(sql), con)

## Q1

In [4]:
sql = """
select count(*)
from 
  country co
  join city ci on ci.partofcountryid = co.id
  join person p on p.locationcityid = ci.id
  join forum_hasmember_person fmbr on fmbr.personid = p.id
  join forum f on f.id = fmbr.forumid
  join message msgp on msgp.containerforumid = f.id
  join message msgc on msgc.parentmessageid = msgp.id
  join message_hastag_tag mtag on mtag.messageid = msgc.id
  join tag t on t.id = mtag.tagid
  join tagclass tc on tc.id = t.typetagclassid
"""
exe(sql) # 155.615.588, 10.5s

,count
0,155615588


## Q2

In [5]:
sql = """
select count(*)
from 
  person p1
  join person_knows_person knows on knows.person1id = p1.id
  join person p2 on p2.id = knows.person2id
  join message msgc on msgc.creatorpersonid = p1.id
  join message msgp on msgp.creatorpersonid = p2.id 
where msgp.parentmessageid is null and msgc.parentmessageid = msgp.id
"""
exe(sql) # 296.644, 0.5s

,count
0,296644


## Q3

In [6]:
sql = """
select count(*)
from 
  country co
  join city  ci_a on ci_a.partofcountryid  = co.id
  join city  ci_b on ci_b.partofcountryid  = co.id
  join city  ci_c on ci_c.partofcountryid  = co.id
  join person  p1 on p1.locationcityid  = ci_a.id
  join person  p2 on p2.locationcityid  = ci_b.id
  join person  p3 on p3.locationcityid  = ci_c.id
  join person_knows_person  knows12 on knows12.person1id = p1.id and knows12.person2id = p2.id
  join person_knows_person  knows13 on knows13.person1id = p1.id and knows13.person2id = p3.id
  join person_knows_person  knows23 on knows23.person1id = p2.id and knows23.person2id = p3.id
"""
exe(sql) # 71.354, 3.5s

,count
0,71354


## Q4

In [7]:
sql = f"""
select count(*)
from
  tag t
  join message_hastag_tag mtag on mtag.tagid = t.id
  join message msg on msg.id = mtag.messageid
  join person p_creator on p_creator.id = msg.creatorpersonid
  join person_likes_message plikes on plikes.messageid = msg.id
  join person p_liker on p_liker.id = plikes.personid 
  join message msgc on msgc.parentmessageid = msg.id
"""
exe(sql) # 9.853.666, 1.5s

,count
0,9853666


## Q5

In [8]:
sql = """
select count(*)
from
  tag t1
  join message_hastag_tag  mtag1 on mtag1.tagid = t1.id
  join message msg on msg.id = mtag1.messageid
  join message msgc on msgc.id = msg.parentmessageid
  join message_hastag_tag mtag2 on mtag2.messageid = msgc.id
  join tag t2 on t2.id = mtag2.tagid
where t2.id <> t1.id
"""
exe(sql) # 9.679.562, 2s

,count
0,9679562


## Q6

In [9]:
sql = """
select count(*)
from 
  person p1
  join person_knows_person knows12 on knows12.person1id = p1.id
  join person p2 on p2.id = knows12.person2id
  join person_knows_person knows23 on knows23.person1id = p2.id
  join person p3 on p3.id = knows23.person2id
  join person_hasinterest_tag pintrst on pintrst.personid = p3.id
  join tag t on t.id = pintrst.tagid
where p3.id <> p1.id
"""
exe(sql) # 176.011.014, 7.6s

,count
0,176011014


## Q7

In [10]:
sql = f"""
select count(*)
from
  tag t
  join message_hastag_tag mtag on mtag.tagid = t.id
  join message msg on msg.id = mtag.messageid
  join person p_creator on p_creator.id = msg.creatorpersonid
  left join person_likes_message plikes on plikes.messageid = msg.id
  join person p_liker on p_liker.id = plikes.personid 
  left join message msgc on msgc.parentmessageid = msg.id
"""
exe(sql) # 11.577.037, 1.5s

,count
0,11577037


## Q8

In [11]:
sql = """
select count(*)
from
  tag t1
  join message_hastag_tag  mtag1 on mtag1.tagid = t1.id
  join message msg on msg.id = mtag1.messageid
  join message msgc on msgc.id = msg.parentmessageid
  join message_hastag_tag mtag2 on mtag2.messageid = msgc.id
  join tag t2 on t2.id = mtag2.tagid
  left join message_hastag_tag mtag3 on mtag3.tagid = t1.id and mtag3.messageid = msgc.id
where t2.id <> t1.id and mtag3.messageid is null
"""
exe(sql) # 5.592.152, 3s

,count
0,5592152


## Q9

In [12]:
sql = """
select count(*)
from 
  person p1
  join person_knows_person knows12 on knows12.person1id = p1.id
  join person p2 on p2.id = knows12.person2id
  join person_knows_person knows23 on knows23.person1id = p2.id
  join person p3 on p3.id = knows23.person2id
  join person_hasinterest_tag pintrst on pintrst.personid = p3.id
  join tag t on t.id = pintrst.tagid
  left join person_knows_person knows13 on knows13.person1id = p1.id and knows13.person2id = p3.id
where p3.id <> p1.id and knows13.person1id is null
"""
exe(sql) # 169.140.023, 7.2s

,count
0,169140023
